In [13]:
import pandas as pd
import sys
sys.path.append('.')  # Добавляем текущую директорию в путь поиска модулей
from utils import search_history
from utils import make_recommendations


user_actions = search_history(user_id=666, user_type=3)
homes = pd.read_csv('data/homes_fixed_scaled.csv')
recs = make_recommendations(homes, user_actions)

utils.py загружен


/Users/andrey_kiseleq/Documents/vs_code/itmo/rec_systems/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

divide by zero encountered in matmul

/Users/andrey_kiseleq/Documents/vs_code/itmo/rec_systems/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

overflow encountered in matmul

/Users/andrey_kiseleq/Documents/vs_code/itmo/rec_systems/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

invalid value encountered in matmul



In [18]:
final_recommendations = recs
homes_fixed = homes.copy()

In [19]:
# Создадим структуру для хранения результатов тестов
stat_test_results = {}

# Выполнение t-теста для каждого пользователя
for user in final_recommendations.keys():
    interacted_obj_ids = user_actions[user_actions['user_id'] == user]['obj_id']
    recommended_obj_ids = final_recommendations[user]

    interacted_objects = homes_fixed.loc[homes_fixed.index.intersection(interacted_obj_ids)]
    recommended_objects = homes_fixed.loc[homes_fixed.index.intersection(recommended_obj_ids)]

    if not interacted_objects.empty and not recommended_objects.empty:
        # Средние значения по объектам
        interacted_mean = interacted_objects.mean(axis=0)
        recommended_mean = recommended_objects.mean(axis=0)

        # Проведение t-теста
        t_stat, p_val = ttest_ind(interacted_mean, recommended_mean, equal_var=False, nan_policy='omit')

        # Сохранение результатов
        stat_test_results[user] = {'t_stat': t_stat, 'p_val': p_val}

        # Вывод результатов для каждого пользователя
        print(f'User {user}: t_stat = {t_stat:.5f}, p_val = {p_val:.5f}')

# Преобразуем в DataFrame для удобного анализа
stat_test_results_df = pd.DataFrame.from_dict(stat_test_results, orient='index')

# Подсчет доли пользователей, у которых p_val > 0.05
no_significant_diff_ratio = (stat_test_results_df['p_val'] > 0.05).mean()
print(f'Доля пользователей, у которых нет статистически значимой разницы: {no_significant_diff_ratio:.2%}')


User 666: t_stat = -0.00778, p_val = 0.99390
Доля пользователей, у которых нет статистически значимой разницы: 100.00%


если p_val > 0.05, то нет оснований полагать, что есть стат. значимая разница => рекомендации были хорошими для всех юзеров